In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import sys
import re
import pygsheets
import os.path 

In [19]:
city_input = input("City: ")

City: Mexico City


In [20]:
city = city_input.replace(' ', '-').lower()

In [4]:
# URLs of page to be scraped
conde_nast_restaurants_url = 'https://www.cntraveler.com/gallery/best-restaurants-in-{0}'.format(city)
conde_nast_activities_url = 'https://www.cntraveler.com/gallery/best-things-to-do-in-{0}'.format(city)

### Conde Nast Restaurants Retrieval

In [5]:
response = requests.get(conde_nast_restaurants_url)

In [6]:
soup = bs(response.text,'html.parser')

In [7]:
results = soup.find_all('div', class_="image-content-container")

if len(results) == 0:
    print("No data found for " + city_input )
    sys.exit()
else: pass

In [8]:
restaurant_list = []
price_list = []
blurb_list = []

for result in results:
    try:
        restaurant = result.find('h2', class_='hed').text
        price = result.find('p', class_='price').text
        blurb = result.find(class_='dek').text
        if (restaurant and price and blurb):
            restaurant_list.append(restaurant)
            price_list.append(price)
            blurb_list.append(blurb)
    except AttributeError as e:
        print(e)

In [9]:
restaurants = pd.DataFrame(list(zip(restaurant_list, price_list, blurb_list)), 
               columns =['Restaurant', 'Price', 'Description'])

### Conde Nast Activity Retrieval

In [10]:
response2 = requests.get(conde_nast_activities_url)
soup2 = bs(response2.text,'html.parser' )

In [11]:
results = soup2.find_all('div', class_="image-content-container")

if len(results) == 0:
    print("No data found for "+ city_input)
    sys.ext()
else: pass

In [12]:
activity_list = []
blurb_list = []

for result in results:
    try:
        activity = result.find('h2', class_='hed').text
        blurb = result.find(class_='dek').text
        if (activity and blurb):
            activity_list.append(activity)
            blurb_list.append(blurb)
    except AttributeError as e:
        print(e)

In [13]:
activities = pd.DataFrame(list(zip(activity_list, blurb_list)), 
               columns =['Activity', 'Description'])

### Export Dataframes to Google Sheet

In [47]:
def df_to_gsheet(df, city_input, df_name):
    credentials = os.path.expanduser('~/New Documents/Misc/Credentials/cn-travel-planner-0746c5316afb.json') 
    gc = pygsheets.authorize(service_file=credentials)
    
    sh = gc.open('Trips')
    wks = sh.add_worksheet(city_input+'-'+df_name,rows=50,cols=6)
    wks.set_dataframe(df,(1,1))
    wks.frozen_rows = 1
    wks.cell('A1').set_text_format('bold', True)
    wks.cell('B1').set_text_format('bold', True)
    wks.cell('C1').set_text_format('bold', True)

In [48]:
df_to_gsheet(restaurants, city_input, "Restaurants")
df_to_gsheet(activities, city_input, "Activities")

Updated spreadsheet here: https://docs.google.com/spreadsheets/d/1sKBiI8cgU-udIGJpLqoPivKVqCIsxN3Q6WW2H2N5KFM
Updated spreadsheet here: https://docs.google.com/spreadsheets/d/1sKBiI8cgU-udIGJpLqoPivKVqCIsxN3Q6WW2H2N5KFM
